In [3]:
import numpy as np
import pickle
import os

In [4]:
print(os.getcwd())
with open('cvscores', 'r') as f:
    data = pickle.load(f)

/media/Datos/TrabajoLinux/Teleco/Ahora/TFG/catkin_ws/src/recibe_datos


In [5]:
print(data)

[[23.655105045863561, 0.11007091068491644, 409776.32780612243, 410.8757218341438], [22.85692798848055, 0.24107989303919733, 280730.89668367349, 282.06533657774634], [26.919233088590662, 0.15618629814410695, 298996.61607142858, 300.63317310566805], [24.262676706119461, 0.099839650246561787, 264443.4445153061, 266.2301803900271], [24.843191782633465, 0.17729138831297556, 366402.44791666669, 367.51103719075519], [21.665524164835613, 0.1147775116066138, 430709.85416666669, 431.54250081380206], [24.924435933430988, 0.21512982745965323, 316851.91666666669, 318.3331298828125], [21.628452301025391, 0.31993447740872699, 314415.98958333331, 315.78077189127606], [21.353412628173828, 0.29088989893595379, 297640.40625, 299.15697224934894], [23.326742172241211, 0.18486459304889044, 271055.8125, 272.66412353515625]]


In [6]:
print(np.mean([d[1] for d in data]))

0.191006444889
